In [42]:
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxa
import py3dep
import shapely
from shapely.geometry import box
from pathlib import Path
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

In [30]:
ds = xr.open_dataset('/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/052_2021-02-03_2021-02-10.nc')
ds = ds['cor'].sel(band = 'VV').coarsen(x = 100, boundary = 'pad').mean().coarsen(y = 100, boundary = 'pad').mean()

geom = ds.rio.bounds()
hs = py3dep.get_map("Hillshade Gray", geom, resolution=30, geo_crs="epsg:4326", crs="epsg:4326")

states = gpd.read_file('/bsuhome/zacharykeskinen/uavsar-validation/data/states/tl_2021_us_state.shp')

insitu = pd.read_parquet('/bsuhome/zacharykeskinen/uavsar-validation/data/insitu/all_insitu.parq')
insitu = gpd.GeoDataFrame(insitu, geometry = gpd.points_from_xy(insitu.lon, insitu.lat))
insitu = insitu.set_crs('EPSG:4326')
gdf = insitu

lidars = []
lidar_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar')
for lidar_fp in lidar_dir.glob('*.nc'):
    if '.all' in lidar_fp.name or '.sd' in lidar_fp.name:
        continue

    lidar = xr.open_dataset(lidar_fp).isel(time = 0)
    lidar = lidar.rio.reproject_match(hs)
    lidars.append(lidar['lidar-sd'])

In [ ]:
f, ax = plt.subplots(figsize=(12, 10))

# MAKE INSET MAP
ax2 = inset_axes(ax, "35%", "35%", loc="upper left")
geo_states = states.to_crs(epsg = 3857)
geo_states[(geo_states['NAME'] != 'Hawaii') & (geo_states['NAME'] != 'Puerto Rico')].plot(facecolor = 'lightgrey', edgecolor = 'black', ax= ax2)
ID = geo_states[geo_states['NAME'] == 'Idaho'].geometry
ID.plot(facecolor = 'darkgrey', edgecolor = 'black', ax = ax2)
ID_shift = ID.apply(lambda x: shapely.affinity.translate(x, xoff=-1.5e6, yoff=-2.6e6))
ID_shift = ID_shift.apply(lambda x: shapely.affinity.scale(x, xfact= 6, yfact = 6))
ID_shift.plot(ax = ax2)
ax2.set_xlim([-1.7e7, -7400000])
ax2.set_ylim([-0.05e7, 6500000])
ax2.xaxis.set_visible(False)
ax2.yaxis.set_visible(False)
shift = gdf.geometry.to_crs(epsg = 3857)
shift = shift.geometry.apply(lambda x: shapely.affinity.translate(x, xoff=-1.8e6, yoff=-4e6))
geom = gpd.GeoSeries(shift).total_bounds
geom = tuple(geom)
geom = [x + y for x, y in zip(list(geom), [-0.1e6, -.1e6, .1e6, .1e6])]
bbox = box(*geom)
gpd.GeoSeries(bbox).plot(ax = ax2, linewidth = 2, facecolor = 'red', edgecolor = 'black')
ax2.axis('off')

# MAKE MAIN MAP

# plot hill shading
hs.plot(ax = ax, zorder = 1, cmap = 'Greys', alpha = 1, add_colorbar = False)

# plot insitu data
insitu.loc[insitu.datasource == 'SnowEx'].drop_duplicates('site_name').plot(ax = ax, zorder = 4, color = 'black', marker = 'x', label = 'Snowpits')

insitu.loc[(insitu.site_name != 'jackson') & (insitu.datasource == 'NRCS')].drop_duplicates('site_name').plot(ax = ax, zorder = 4, color = 'blue', marker = 'o', label = 'Snotels')

for lidar in lidars:
    lidar.where(lidar.isnull(), 1).plot(ax = ax, add_colorbar = False, cmap = 'Purples', zorder = 3, alpha = 0.75, vmin = 0, vmax = 1)

# plot uavsar bounding box
ds.where(ds.isnull(), 1).plot(alpha = 0.35, cmap = 'Reds', add_colorbar = False, ax = ax, zorder = 3)

# plot stamen topo map
ctx.add_basemap(ax, crs = 'EPSG:4326', source = ctx.providers.Stamen.Terrain, alpha = .65, zorder = 2)


ax.set(xlabel = None)
ax.set(ylabel = None)

# North Arrow
x, y, arrow_length = 0.97, 0.14, 0.05
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='lightgrey', width=3, headwidth=8),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes, color = 'white')

# scale bar
fontprops = fm.FontProperties(size=18)
scalebar = AnchoredSizeBar(ax.transData,
                           0.1, '10 km', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=0.001,
                           fontproperties=fontprops)
ax.add_artist(scalebar)

# legend
uavsar_lg = mpatches.Patch(color='orange', alpha = 0.5, label='UAVSAR Flight Box')
lidar_lg = mpatches.Patch(color='purple', alpha = 0.65, label='LiDAR Flight Boxes')
snotel_lg = mlines.Line2D([], [], color='blue', marker='o', linestyle='None', label='Snotels')
snowpit_lg = mlines.Line2D([], [], color='black', marker='x', linestyle='None', label='Snowpits')

ax.legend(handles=[uavsar_lg, lidar_lg, snotel_lg, snowpit_lg], loc = 'upper right')

# site names
for i, r in insitu.loc[(insitu.site_name != 'jackson') & (insitu.datasource == 'NRCS')].drop_duplicates('site_name').iterrows():
    bx, by = (0.03, 0.01)
    if r.site_name == 'mores':
        bx, by = (0.0145, 0.03)
    if r.site_name == 'banner':
        bx, by = (-0.12, 0)
    ax.text(x = r.lon + bx, y = r.lat + by, s = r.site_name.capitalize(), weight='bold')
plt.savefig('/bsuhome/zacharykeskinen/uavsar-validation/figures/study_area/study_area_map_v1.png')